# Test search functions

To install necessary packages, run `pip install -e '.[search_backend, dev]'`.

Before running this notebook, set up an Opensearch container following instructions here: https://docs.haystack.deepset.ai/v2.0/docs/opensearchbm25retriever

Or alternatively open Docker desktop and run:
```
docker pull opensearchproject/opensearch:2.11.0
docker run -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" -e "OPENSEARCH_JAVA_OPTS=-Xms1024m -Xmx1024m" opensearchproject/opensearch:2.11.0
```

Hybrid search was introduced in OpenSearch v2.11. Not clear whether Haystack is able to properly use a version this recent. Proper hybrid search with OpenSearch hasn't been enabled yet in Haystack.

In [ ]:
import json
import os
# os.chdir('~/rd-search-backend')

from search_backend.api.lib.config import get_config
from search_backend.api.lib.services import SERVICES
from search_backend.api.lib.opensearchpipeline import run_indexing_pipeline, RetrievalPipeline
from search_backend.api.lib.searchfunctions import bm25_search, semantic_search, hybrid_search, pretty_print_results

cfg = get_config()

test_query = "short custody"

print(test_query)

## Read data

In [ ]:
with open('ai_catalogue.json') as f:
    project_list = json.load(f)

print(project_list)

In [3]:
# Replace newlines as they interfere with the matching
project_list = [{k : v.replace('\n', ' ') if v is not None else v for k, v in project.items()} for project in project_list]

In [ ]:
project_list[0]

In [4]:
fields_to_search = [
    'project_name',
    'description',
    'what_does_this_initiative_do',
    'reasons_for_use',
    'problem_solved_by_the_initiative',
    'metrics_or_intended_impacts'
]

In [5]:
def format_doc_dict(doc: dict, field: str):

    """
    Reformat data into format accepted by Haystack.
    Here we wish to search over multiple fields, so we include the text from different
    fields in separate dictionaries within a list.

    :doc: dictionary containing text data to search along with accompanying metadata
    :field: string corresponding to one of the dictionary keys, to indicate the field to index the text from
    """

    content = doc[field]
    # doc.pop(field)

    if content is None:
        # We can't index None values, so returning None here allows us to skip fields where no info is provided
        return None
    else:
        meta = doc.copy()
        meta['matched_field'] = field

        dict = {
            'meta': meta,
            'content': content,
        }

        return dict


dataset = [y for project in project_list[:3] for field in fields_to_search if (y := format_doc_dict(project, field)) is not None]

In [ ]:
print(len(dataset))
dataset

## Set up Opensearch

In [7]:
# Connect to an existing Opensearch document store
# query_document_store = SERVICES["querydocumentstore"]
from haystack_integrations.document_stores.opensearch import OpenSearchDocumentStore
query_document_store = OpenSearchDocumentStore(
    hosts="http://localhost:9200",
    use_ssl=True,
    verify_certs=False,
    http_auth=("admin", "admin"),
    embedding_dim=384,
    recreate_index=True,
    index="document",
)

In [ ]:
run_indexing_pipeline(dataset, query_document_store, cfg, semantic=False)

## Run search

In [9]:
bm25_pipeline = RetrievalPipeline(query_document_store)
bm25_pipeline = bm25_pipeline.setup_bm25_pipeline()

In [ ]:
test_query = "improved service quality"
results = bm25_search(test_query, bm25_pipeline, top_k=3)
pretty_print_results(results["bm25_retriever"]['documents'])

In [ ]:
results["bm25_retriever"]['documents'][0].meta